In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-large')

model = T5ForConditionalGeneration.from_pretrained('t5-large')

# input = "My name is Azeem and I live in India"
# input = "It has never been easy to have a rational conversation about the value of gold."
input = "The horse rides the astronaut. "
# You can also use "translate English to French" and "translate English to Romanian"
input_ids = tokenizer("translate English to German: "+input, return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded)

In [15]:
## Convert SGM to txt file
import re
path='./sgm/newstest2019-ende-src.en.sgm'#'./sgm/newstest2019-ende-ref.de.sgm'#'./sgm/newstest2019-deen-ref.en.sgm'
text_out='newstest2019-ende-src.txt'#'newstest2019-ende-ref.txt'#'newstest2019-deen-ref.txt'
seg_re = re.compile(r"<seg id=\"\d+\">(.*)</seg>")
with open(text_out,'w', encoding="utf-8") as g:
    with open(path, encoding="utf-8") as f:
        for line in f:
            seg_match = re.match(seg_re, line)
            if seg_match:
                assert len(seg_match.groups()) == 1
            else:
                continue
            out_line= seg_match.groups()[0]
            g.write(out_line)
            g.write('\n')

In [16]:
## Translate txt from de to en
# new_t = '/home/mbastan/.local/lib_old/python3.9_old/site-packages/transformers'
from transformers import FSMTForConditionalGeneration, FSMTTokenizer
mname = "facebook/wmt19-en-de"#"facebook/wmt19-de-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)
fair_out_path = 'newstest2019-ende-out.txt'
with open(text_out, encoding="utf-8") as f:
    lines = f.readlines()
with open(fair_out_path,'w', encoding="utf-8") as g:
    for input in lines:
        input_ids = tokenizer.encode(input.strip(), return_tensors="pt")
        outputs = model.generate(input_ids)
        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # print(decoded)
        g.write(decoded)
        g.write('\n')
        #  # Machine learning is great, isn't it?

2023-01-18 18:53:30.248808: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


ImportError: cannot import name 'FSMTForConditionalGeneration' from 'transformers' (/home/mbastan/anaconda3/envs/hug3/lib/python3.7/site-packages/transformers/__init__.py)

In [ ]:
## evaluate the output
from sacrebleu import corpus_bleu
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    with open(score_path,"w") as score_file:
        score_file.write(result)
with open('newstest2019-deen-out.txt') as output:
    output_lns=output.readlines()
with open('newstest2019-deen-ref.txt') as reflns:
    refs_lns=reflns.readlines()
    
calculate_bleu_score(output_lns, refs_lns, './blueScore')
    
    

In [6]:
## extract structures
import stanza
from typing import List
from stanza_batch import batch
import json
from nltk.corpus import stopwords
import pickle
path1='newstest2019-deen-in.txt'
# path1='/home/mbastan/context_home/structuralDecoding/a_star_neurologic/dataset/machine_translation/newstest2017-iate/iate.414.terminology.tsv.de'

pkname='newstest2019'
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"
nlp = stanza.Pipeline('de', processors='lemma,tokenize,pos,depparse',use_gpu=True)
with open(path1) as f:
    lines = f.readlines()
try:
    with open('stanza_documents_%_de.pk'%pkname,'rb') as f:
        stanza_documents=pickle.load(f)
except:
    ## batch document processing to increase the speed
    stanza_documents: List[stanza.Document] = []
    for document in batch(lines, nlp, batch_size=64): # Default batch size is 32
        stanza_documents.append(document)
    ## save the documents on a file
    with open('stanza_documents_%_de.pk'%pkname,'wb') as f:
        pickle.dump(stanza_documents,f)  


2023-01-18 10:53:55 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-01-18 10:53:55 WARNING: Language de package default expects mwt, which has been added
2023-01-18 10:53:56 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |

2023-01-18 10:53:56 INFO: Use device: gpu
2023-01-18 10:53:56 INFO: Loading: tokenize
2023-01-18 10:53:56 INFO: Loading: mwt
2023-01-18 10:53:56 INFO: Loading: pos
2023-01-18 10:53:56 INFO: Loading: lemma
2023-01-18 10:53:56 INFO: Loading: depparse
2023-01-18 10:53:56 INFO: Done loading processors!


ValueError: unsupported format character '_' (0x5f) at index 18

In [7]:
from deep_translator import PonsTranslator
translator = PonsTranslator(source='de', target='en')
outpath='newstest2019_deen_en.constraint.json' #'iate.414_en.terminology.constraint.json'
with open(outpath,'w') as f:
    stop_words = set(stopwords.words('english'))
    for doc in  stanza_documents:
        sent = doc.sentences[0]
        arr_all = []
        arr_ori = []
        arr_rev = []
        arr_lex =[]
        sent_map ={}
        for word in sent.words:
            arr_all.append([word.text,word.deprel])
            # if word.deprel in sent_map:
            #     continue
            if word.deprel == 'nsubj' or word.deprel == 'obj' or word.deprel == 'root' or word.deprel == 'obl':
                if( word.deprel == 'obl' and 'obj' in sent_map) or ( word.deprel == 'obj' and 'obl' in sent_map):
                    continue
                
                # ids = tokenizer.encode(word.text.strip(), return_tensors="pt")
                # outs = model.generate(ids,do_sample=True, top_k=30, top_p=0.95)
                # decoded = tokenizer.decode(outs[0], skip_special_tokens=True)
                try:
                    translated_word=translator.translate(word.text)
                except:
                    translated_word=word.text
                translated_nlp=nlp(translated_word)
                translated_lemma = [w.lemma for translated_sent in translated_nlp.sentences \
                    for w in translated_sent.words\
                        if w.lemma.lower() not in stop_words and w.text not in stop_words]
                for tlemma in translated_lemma:                
                    arr_ori.append([tlemma,word.deprel])


        ## write all structures
        json.dump(arr_ori,f)
        f.write('\n')
        
        # json.dump(arr_ori,g_ori)
        # g_ori.write('\n')
        
        # json.dump(arr_rev,g_rev)
        # g_rev.write('\n')
        
        # json.dump(arr_lex,f_lexica_constraints)
        # f_lexica_constraints.write('\n')

In [12]:
### compare translation outputs

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def cosine_similarity(X,Y):
        # tokenization
        X_list = word_tokenize(X) 
        Y_list = word_tokenize(Y)
        
        # sw contains the list of stopwords
        sw = stopwords.words('english') 
        l1 =[];l2 =[]
        
        # remove stop words from the string
        X_set = {w for w in X_list if not w in sw} 
        Y_set = {w for w in Y_list if not w in sw}
        
        # form a set containing keywords of both strings 
        rvector = X_set.union(Y_set) 
        for w in rvector:
            if w in X_set: l1.append(1) # create a vector
            else: l1.append(0)
            if w in Y_set: l2.append(1)
            else: l2.append(0)
        c = 0
        
        # cosine formula 
        for i in range(len(rvector)):
                c+= l1[i]*l2[i]
        cosine = c / float((sum(l1)*sum(l2))**0.5)
        # print("similarity: ", cosine)
        
        return(cosine)
        
constraints=open('newstest2019_deen.constraint.json').readlines()
sd_out=open('SD_out_deen_v3.out').readlines()
nd_out=open('SD_out_deen_neurologic_deocding.out').readlines()
fairseq_out=open('newstest2019-deen-out.txt').readlines()
ref=open('newstest2019-deen-ref.txt').readlines()

import pandas as pd

rows=[]
for i,(cons,sd,nd,fair,re) in enumerate(zip(constraints,sd_out,nd_out,fairseq_out,ref)):
    sd_nd_cosine=cosine_similarity(sd,nd)
    sd_re_cosine=cosine_similarity(sd,re)
    row={
        'ind':i,
        'constrains':cons,
        'structura_decoding':sd,
        'neurological_decoding':nd,
        'fairseq':fair,
        'target':re,
        'sd_nd_cosine':sd_nd_cosine,
        'sd_re_cosine':sd_re_cosine,
        'sd_nd_diff':sd_re_cosine-sd_nd_cosine
    }
    rows.append(row)
    
df=pd.DataFrame(rows)
df.to_csv('translation_out.csv',index=False)

In [9]:
df[:5]

,ind,constrains,structura_decoding,neurological_decoding,fairseq,target
0,0,"[[""Munich"", ""root""]]\n",Beautiful Munich girl 2018: Beautiful Munich w...,Beautiful Munich girl 2018: Beautiful Munich w...,Beautiful Munich 2018: Beautiful Munich 2018 i...,The Beauty of Munich 2018: the Beauty of Munic...
1,1,"[[""www.ecs-webhosting.de"", ""root""], [""04.05.20...","By az, updated on 04.\n","By az, updated on 04.05.2018 at 11:11\n","By az, updated on 04 / 05 / 2018 at 11: 11\n","From A-Z, updated on 04/05/2018 at 11:11\n"
2,2,"[[""she"", ""nsubj""], [""to"", ""root""], [""want"", ""r...","Yeah, she wants...\n","Yeah, she wants...\n","Yes, she wants...\n","Yes, she wants to...\n"
3,3,"[[""2018"", ""obl""], [""he's"", ""root""], [""just"", ""...","""Beautiful Munich"" 2018 will be!\n","""Beautiful Munich"" 2018 will be!\n","Become a ""beautiful Munich woman"" in 2018!\n","to become ""The Beauty of Munich"" in 2018!\n"
4,4,"[[""afternoon"", ""obl""], [""to"", ""root""], [""wait""...","In the afternoon, another surprise awaits our ...","In the afternoon, another surprise awaits our ...",In the afternoon another surprise awaits our c...,In the afternoon there is another surprise wai...
